# Managing and Cleaning Up Endpoints

This lesson focuses on the essential management tasks that every SageMaker practitioner must master: checking endpoint status and configuration details, then properly cleaning up resources to avoid ongoing charges.

This lesson will teach you two complementary approaches to endpoint management: `programmatic management` using Python and the SageMaker SDK, and `command-line management` using the AWS CLI. The Python approach provides detailed programmatic access that's ideal for automation and integration into larger workflows, while the AWS CLI offers quick, interactive commands perfect for manual inspection and cleanup tasks. By mastering both approaches, you'll have the flexibility to choose the most appropriate tool for each management scenario you encounter.


## Checking Endpoint Status

The following Python script demonstrates how to retrieve and display the status of all your SageMaker endpoints. This approach builds on the SageMaker session concepts you've used throughout previous lessons, but now focuses on management rather than deployment operations.



In [ ]:
import sagemaker

# Create a SageMaker session 
sagemaker_session = sagemaker.Session()

# Get list of all endpoints from SageMaker
endpoints_response = sagemaker_session.sagemaker_client.list_endpoints()

# Extract the endpoints array from the response
endpoints = endpoints_response['Endpoints']

# Print the name of each endpoint
print("SageMaker Endpoints:")
for endpoint in endpoints:
    print(f"- {endpoint['EndpointName']} ({endpoint['EndpointStatus']})")

The response contains an array of endpoint objects, each including much more data than just the endpoint name and current status shown in this example. Each endpoint object also contains the `EndpointArn`, `CreationTime`, `LastModifiedTime`, and other metadata that provides comprehensive information about your deployed resources.



### Checking Endpoint Status with AWS CLI

The AWS CLI provides an equivalent command for quickly checking endpoint status from the command line. This approach is particularly useful for quick status checks or when working in environments where you prefer command-line tools over Python scripts.


In [ ]:
# List all SageMaker endpoints
aws sagemaker list-endpoints

This command produces a JSON output that contains the same information as the Python approach, but in a format that's easy to read for manual inspection:


```
{
    "Endpoints": [
        {
            "EndpointName": "california-housing-modeltrainer",
            "EndpointArn": "arn:aws:sagemaker:us-east-1:605926690821:endpoint/california-housing-modeltrainer",
            "CreationTime": "2025-07-28T08:18:29.553000+00:00",
            "LastModifiedTime": "2025-07-28T08:20:57.772000+00:00",
            "EndpointStatus": "InService"
        },
        {
            "EndpointName": "california-housing-estimator",
            "EndpointArn": "arn:aws:sagemaker:us-east-1:605926690821:endpoint/california-housing-estimator",
            "CreationTime": "2025-07-28T08:18:17.051000+00:00",
            "LastModifiedTime": "2025-07-28T08:20:55.435000+00:00",
            "EndpointStatus": "InService"
        },
        ...
    ]
}
```

## Inspecting Endpoint Configuration Details

Beyond basic status information, effective endpoint management requires understanding the detailed configuration of your deployed endpoints. This includes information about the underlying compute resources, scaling settings, and model configurations that determine how your endpoints operate and what they cost. 

In [ ]:
import json

# Iterate through each endpoint and get configuration details
for endpoint in endpoints:
    # Get endpoint name
    endpoint_name = endpoint['EndpointName']
    
    # Display endpoint name
    print(f"Configuration for {endpoint_name}")
    
    # Get endpoint details to retrieve the configuration name
    endpoint_details = sagemaker_session.describe_endpoint(endpoint_name)

    # Get endpoint configuration name
    config_name = endpoint_details['EndpointConfigName']
    
    # Get endpoint configuration details
    config_details = sagemaker_session.sagemaker_client.describe_endpoint_config(
        EndpointConfigName=config_name
    )
    
    # Display configuration as JSON
    print(json.dumps(config_details, indent=2, default=str))

When you run this script, you'll see detailed output like this for each endpoint:


```
Configuration for california-housing-realtime
{
  "EndpointConfigName": "california-housing-realtime",
  "EndpointConfigArn": "arn:aws:sagemaker:us-east-1:605926690821:endpoint-config/california-housing-realtime",
  "ProductionVariants": [
    {
      "VariantName": "AllTraffic",
      "ModelName": "sagemaker-scikit-learn-2025-07-28-08-15-32-100",
      "InitialInstanceCount": 1,
      "InstanceType": "ml.m5.large",
      "InitialVariantWeight": 1.0
    }
  ],
  "CreationTime": "2025-07-28T08:15:33.670000+00:00",
  "EnableNetworkIsolation": false
}
```


The configuration details reveal crucial information about how your endpoints are deployed. For serverless endpoints, you'll see `ServerlessConfig` sections that specify memory allocation and maximum concurrency settings. For real-time endpoints, you'll see `InitialInstanceCount` and `InstanceType` specifications that determine the persistent compute resources and their associated costs.



In [ ]:
# Describe an endpoint
aws sagemaker describe-endpoint --endpoint-name california-housing-realtime

# Describe an endpoint config
aws sagemaker describe-endpoint-config --endpoint-config-name california-housing-realtime

## Deleting Endpoints and Their Associated Resources


When you no longer need an endpoint, you must delete both the endpoint itself and its associated endpoint configuration.

Notice that the endpoint must be deleted before its configuration can be removed, as SageMaker prevents deletion of configurations that are still in use by active endpoints.

In [ ]:
# Name of the SageMaker endpoint we want to delete
ENDPOINT_NAME = "california-housing-local-model"

# Get endpoint details before deletion
endpoint_details = sagemaker_session.describe_endpoint(ENDPOINT_NAME)

# Retrieve the associated config name
config_name = endpoint_details['EndpointConfigName']

# Delete the endpoint first - this stops the running instances/serverless compute
# The endpoint must be deleted before its configuration can be removed
sagemaker_session.delete_endpoint(ENDPOINT_NAME)

# Delete the endpoint configuration to avoid leaving orphaned configs
# This removes the blueprint that defined the endpoint's compute resources
sagemaker_session.delete_endpoint_config(config_name)

In [ ]:
# Delete an endpoint
aws sagemaker delete-endpoint --endpoint-name california-housing-realtime

# Delete an endpoint config
aws sagemaker delete-endpoint-config --endpoint-config-name california-housing-realtime